In [2]:
import csv
import requests
import pandas as pd

# Function to get lyrics from the API
def get_lyrics(artist, title):
    url = f"https://api.lyrics.ovh/v1/{artist}/{title}"
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        data = response.json()

        lyrics = data.get('lyrics', 'Lyrics not found')
        lyrics = str(lyrics)


        escaped_text = lyrics.replace("\n", "\\n").strip()
        return escaped_text
    except Exception as e:
        return ""

# Read the CSV file into a pandas DataFrame
input_file = 'processed_songs_original.csv'
output_file = 'processed_songs_with_lyrics'

df = pd.read_csv(input_file)

# Get lyrics for each song and add to the DataFrame, then save to a new CSV file for every 50 songs, each csv contain different 50 songs

for i in range(0, len(df), 50):
    chunk = df.iloc[i:i+50]

    # Apply the get_lyrics function to the chunk
    chunk['lyrics'] = chunk.apply(lambda x: get_lyrics(x['Artists'], x['Song Name']), axis=1)
    
    # Save the chunk to a CSV file
    chunk.to_csv(f"{output_file}_{i}.csv", index=False)

# print(df.head())


/var/folders/8j/ztcwr_dj3kq17rqj6tyrrf5r0000gn/T/ipykernel_56050/504040300.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['lyrics'] = chunk.apply(lambda x: get_lyrics(x['Artists'], x['Song Name']), axis=1)
/var/folders/8j/ztcwr_dj3kq17rqj6tyrrf5r0000gn/T/ipykernel_56050/504040300.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['lyrics'] = chunk.apply(lambda x: get_lyrics(x['Artists'], x['Song Name']), axis=1)
/var/folders/8j/ztcwr_dj3kq17rqj6tyrrf5r0000gn/T/ipykernel_56050/504040300.p

In [10]:
import pandas as pd
import glob

# Path to the folder containing the CSV files
input_folder = "."

# Pattern to match the files (e.g., all files ending with .csv)
file_pattern = f"{input_folder}/*_*.csv"

# List all matching files
csv_files = glob.glob(file_pattern)

# Read and concatenate all CSV files
combined_df = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)

# Save the combined DataFrame to a single CSV file
output_file = "processed_songs_with_lyrics.csv"
combined_df.to_csv(output_file, index=False)

print(f"Combined file saved as {output_file}")

Combined file saved as processed_songs_with_lyrics.csv


In [11]:
file = "processed_songs_with_lyrics.csv"
df = pd.read_csv(file)

filtered_df = df[~df['lyrics'].isna()]
filtered_df.to_csv("songs_without_lyrics.csv", index=False)

In [3]:
import pandas as pd
file = "processed_songs_original.csv"
df = pd.read_csv(file)

df = df[['Song Name', 'Track_ID']]

df.to_csv("processed_songs_original_01.csv", index=False)

In [48]:
import requests
from bs4 import BeautifulSoup

# Genius API base URL
GENIUS_API_BASE_URL = "https://api.genius.com"
GENIUS_API_KEY = "IKzW2Jaor6tNcL5AyRjL7OdKOR4Pc28f9kVHUXXvsoVjHwh0Rb_MnkW1JyrI7eep"  # Replace with your Genius API key

def get_lyrics_genius(artist, song_name):
    """
    Fetches the lyrics of a song by an artist using the Genius API.
    """
    # Search for the song on Genius
    search_url = f"{GENIUS_API_BASE_URL}/search"
    headers = {"Authorization": f"Bearer {GENIUS_API_KEY}"}
    params = {"q": f"{song_name} {artist}"}
    response = requests.get(search_url, headers=headers, params=params)
    
    if response.status_code != 200:
        return f"Error: Unable to fetch data from Genius API. Status code: {response.status_code}"
    
    data = response.json()

    
    # Check if results were found
    if not data['response']['hits']:
        return "No lyrics found."
    
    # Extract the song's Genius page URL
    song_url = data['response']['hits'][0]['result']['url']
    # Scrape the lyrics from the song's Genius page
    lyrics = scrape_lyrics(song_url)

    return lyrics

def scrape_lyrics(song_url):

    print(song_url)
    """
    Scrapes the lyrics from the Genius song page.
    """
    response = requests.get(song_url)
    if response.status_code != 200:
        return f"Error: Unable to fetch song page. Status code: {response.status_code}"
    
    # Parse the HTML
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find the lyrics container
    lyrics_container = soup.find("div", class_="Lyrics-sc-1bcc94c6-1 bzTABU")
    if lyrics_container:
        return lyrics_container.get_text(separator="\n")
    
    # Alternative method for older Genius pages
    lyrics_container = soup.find("div", class_="lyrics")
    if lyrics_container:
        return lyrics_container.get_text(separator="\n")
    
    return "Lyrics not found."

# Example usage
artist_name = "a-ha"
song_name = "Take on Me"
lyrics = get_lyrics_genius(artist_name, song_name)
print(lyrics)

https://genius.com/A-ha-take-on-me-lyrics
[Verse 1]
We're talking away
I don't know what I'm to say
I'll say it anyway
Today is another day to find you
Shyin' away
I'll be comin' for your love, okay?
[Chorus]
Take on me (Take on me)
Take me on (Take on me)
I'll be gone
In a day or two
[Verse 2]
So, needless to say
I'm odds and ends
But I'll be stumblin' away
Slowly learnin' that life is okay
Say after me
"It's no better to be safe than sorry"
[Chorus]
Take on me (Take on me)
Take me on (Take on me)
I'll be gone
In a day or two


In [52]:
import pandas as pd

def get_lyrics(artist, title):
    url = f"api.genius.com/search?q={title} {artist}"
    try:
        # request with Authorization: Bearer
        response = requests.get(url, headers={'Authorization': 'Bearer ' + 'IKzW2Jaor6tNcL5AyRjL7OdKOR4Pc28f9kVHUXXvsoVjHwh0Rb_MnkW1JyrI7eep'}, timeout=5)

        response.raise_for_status()
        data = response.json()

        # lyrics is in response > hits > [0] > result > url

        lyrics = data.get('response', 'Lyrics not found')
        lyrics = str(lyrics)


        escaped_text = lyrics.replace("\n", "\\n").strip()
        return escaped_text
    except Exception as e:
        return ""


output_file = "processed_songs_with_lyrics.csv"
df = pd.read_csv(output_file)

# Filter rows where the last column ('lyrics') is empty
filtered_df = df[df['lyrics'].isna()]

filter_df_first_five = filtered_df
# Display the filtered DataFrame
filter_df_first_five['lyrics'] = filter_df_first_five.apply(lambda x: get_lyrics_genius(x['Artists'], x['Song Name']), axis=1)

# Export csv 
filter_df_first_five.to_csv('additional_songs_with_lyrics.csv', index=False)

https://genius.com/Sam-harris-neuroscientist-shouldering-the-burden-of-history-11-annotated
https://genius.com/A-ha-take-on-me-lyrics
https://genius.com/Earthside-we-who-lament-lyrics
https://genius.com/Spotify-this-is-ava-max-lyrics
https://genius.com/Genius-brasil-calendario-de-lancamento-de-singles-de-setembro-2023-annotated
https://genius.com/Just-jase-the-friendzone-playlist-annotated
https://genius.com/Shakira-hips-dont-lie-feat-wyclef-jean-live-lyrics
https://genius.com/Knaan-wavin-flag-coca-cola-celebration-mix-lyrics
https://genius.com/Hitradio-o3-nummer-eins-hits-in-osterreich-2013-annotated
https://genius.com/Eagles-eagles-hotel-california-2013-remaster-turkce-ceviri-lyrics
https://genius.com/Whoistiz-2023-listening-log-lyrics
https://genius.com/Sosad2112-sosad2112s-utterly-savage-music-collection-annotated
https://genius.com/Black-sabbath-the-sign-of-the-southern-cross-heaven-and-hell-live-2008-remaster-lyrics
https://genius.com/Sosad2112-sosad2112s-utterly-savage-music-col

/var/folders/8j/ztcwr_dj3kq17rqj6tyrrf5r0000gn/T/ipykernel_98863/1212526.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df_first_five['lyrics'] = filter_df_first_five.apply(lambda x: get_lyrics_genius(x['Artists'], x['Song Name']), axis=1)


In [54]:
process_songs_df = pd.read_csv('processed_songs_with_lyrics.csv')
additional_songs_df = pd.read_csv('additional_songs_with_lyrics.csv')

# Overwrite record in the process_songs_df with songs exist in additional_songs_df, matching by 'Artists' and 'Song Name'
for index, row in additional_songs_df.iterrows():
    process_songs_df.loc[(process_songs_df['Artists'] == row['Artists']) & (process_songs_df['Song Name'] == row['Song Name']), 'lyrics'] = row['lyrics']

# Save the updated DataFrame to a new CSV file
output_file = "processed_songs_with_lyrics_updated.csv"
process_songs_df.to_csv(output_file, index=False)



